In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pprint
import tempfile
import tensorflow as tf

from tfx import v1 as tfx
# import InteractiveContext to enable iterative development
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext


2023-11-09 18:23:23.129902: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-09 18:23:27.574032: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-09 18:23:31.874338: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


In [3]:
pipeline_name = "pikastro"
pipeline_root = "/tmp/artifacts"
print(pipeline_root)


/tmp/artifacts


In [4]:

context = InteractiveContext(pipeline_name=pipeline_name, pipeline_root=pipeline_root)

In [5]:
context.pipeline_root

'/tmp/artifacts'

In [6]:
import tfx.proto.example_gen_pb2 as proto

In [7]:
data_root = "/home/azureuser/cloudfiles/code/Users/stefruinard/Pikastro/src/dynax/data/electricity"

input_config = proto.Input(splits=[
                proto.Input.Split(name='train',
                                            pattern='span-{SPAN}/**/version-{VERSION}/*'),
                proto.Input.Split(name='eval',
                                            pattern='span-{SPAN}/**/version-{VERSION}/*')
            ])

example_gen = tfx.components.CsvExampleGen(input_base=data_root, input_config=input_config)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [8]:
import glob
src = glob.glob(os.path.join(example_gen.outputs["examples"].get()[0].uri, "Split-train/*"))
print(src)
example_dataset = tf.data.TFRecordDataset(src, compression_type="GZIP")
for example in example_dataset.take(5):
    print(tf.train.SequenceExample.FromString(example.numpy()))

['/tmp/artifacts/CsvExampleGen/examples/1/Split-train/data_tfrecord-00000-of-00001.gz']


context {
  feature {
    key: "annual_consume"
    value {
      float_list {
        value: 5524.52001953125
      }
    }
  }
  feature {
    key: "annual_consume_lowtarif_perc"
    value {
      float_list {
        value: 28.40999984741211
      }
    }
  }
  feature {
    key: "city"
    value {
      bytes_list {
        value: "WERKENDAM"
      }
    }
  }
  feature {
    key: "delivery_perc"
    value {
      float_list {
        value: 97.62000274658203
      }
    }
  }
  feature {
    key: "net_manager"
    value {
      bytes_list {
        value: "Enexis B.V."
      }
    }
  }
  feature {
    key: "num_connections"
    value {
      int64_list {
        value: 42
      }
    }
  }
  feature {
    key: "perc_of_active_connections"
    value {
      int64_list {
        value: 100
      }
    }
  }
  feature {
    key: "purchase_area"
    value {
      bytes_list {
        value: "ENEXIS"
      }
    }
  }
  feature {
    key: "smartmeter_perc"
    value {
      float_list

2023-11-09 18:25:24.016203: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [9]:
from tfx.components.example_gen.component import FileBasedExampleGen
from tfx.components.base import executor_spec
from tfx.components.example_gen.csv_example_gen import executor
from tfx.proto import example_gen_pb2


In [10]:
example_gen_pb2.FORMAT_TF_SEQUENCE_EXAMPLE

7

In [11]:
from dynax.components import timeseries_gen

In [29]:
import glob
input_base = "data/electricity/span-2020/"
print(glob.glob(input_base))
example_gen = FileBasedExampleGen(
    input_base=data_root,
    input_config=input_config,
    custom_executor_spec=executor_spec.ExecutorClassSpec(timeseries_gen.Executor),
    output_data_format=example_gen_pb2.FORMAT_TF_SEQUENCE_EXAMPLE,
    custom_config={
        "history_size":10,
        "target_size":1,
        "shift": 1,
        "stride": 1,
        "max_items_to_process": 20,
        "context_features": ['net_manager', 'purchase_area', 'street', 'zipcode_from', 'zipcode_to', 'city', 'num_connections', 'delivery_perc', 'perc_of_active_connections', 'type_conn_perc', 'type_of_connection'],
        "sequence_features": ['annual_consume', 'annual_consume_lowtarif_perc', 'smartmeter_perc'],
        }
    )

['data/electricity/span-2020/']


In [30]:
context.run(example_gen, enable_cache=False)

['/home/azureuser/cloudfiles/code/Users/stefruinard/Pikastro/src/dynax/data/electricity/span-2020/enexis/version-1/data.csv']
{
  "splits": [
    {
      "name": "train",
      "pattern": "span-2020/**/version-1/*"
    },
    {
      "name": "eval",
      "pattern": "span-2020/**/version-1/*"
    }
  ]
}
{"context_features": ["net_manager", "purchase_area", "street", "zipcode_from", "zipcode_to", "city", "num_connections", "delivery_perc", "perc_of_active_connections", "type_conn_perc", "type_of_connection"], "history_size": 10, "max_items_to_process": 20, "sequence_features": ["annual_consume", "annual_consume_lowtarif_perc", "smartmeter_perc"], "shift": 1, "stride": 1, "target_size": 1}
['/home/azureuser/cloudfiles/code/Users/stefruinard/Pikastro/src/dynax/data/electricity/span-2020/enexis/version-1/data.csv']
{
  "splits": [
    {
      "name": "train",
      "pattern": "span-2020/**/version-1/*"
    },
    {
      "name": "eval",
      "pattern": "span-2020/**/version-1/*"
    }
  

ExecutionResult(
    component_id: FileBasedExampleGen
    execution_id: 8
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=FileBasedExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [31]:
src = glob.glob(os.path.join(example_gen.outputs["examples"].get()[0].uri, "Split-train/*"))
print(src)


['/tmp/artifacts/FileBasedExampleGen/examples/8/Split-train/data_tfrecord-00000-of-00001.gz']


In [32]:
example_dataset = tf.data.TFRecordDataset(src, compression_type="GZIP")
for example in example_dataset.take(1):
    print(tf.train.SequenceExample.FromString(example.numpy()))

context {
  feature {
    key: "city"
    value {
      bytes_list {
        value: "WERKENDAM"
      }
    }
  }
  feature {
    key: "delivery_perc"
    value {
      float_list {
        value: 97.62000274658203
      }
    }
  }
  feature {
    key: "net_manager"
    value {
      bytes_list {
        value: "Enexis B.V."
      }
    }
  }
  feature {
    key: "num_connections"
    value {
      int64_list {
        value: 42
      }
    }
  }
  feature {
    key: "perc_of_active_connections"
    value {
      int64_list {
        value: 100
      }
    }
  }
  feature {
    key: "purchase_area"
    value {
      bytes_list {
        value: "ENEXIS"
      }
    }
  }
  feature {
    key: "street"
    value {
      bytes_list {
        value: "Sasdijk"
      }
    }
  }
  feature {
    key: "type_conn_perc"
    value {
      float_list {
        value: 50.0
      }
    }
  }
  feature {
    key: "type_of_connection"
    value {
      bytes_list {
        value: "3x25"
      }
    }


In [33]:
from tfx.components import StatisticsGen
from tfx.components import SchemaGen
from tfx.components import ExampleValidator
from tfx.components import Transform
from tfx.components import Trainer
from tfx.components import Evaluator
from tfx.components import Pusher
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.components.trainer.executor import GenericExecutor

In [34]:
stats = StatisticsGen(examples=example_gen.outputs["examples"])
context.run(stats)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 9
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [35]:
# visualize stats

context.show(stats.outputs['statistics'])

In [37]:
schema_gen = SchemaGen(statistics=stats.outputs["statistics"], infer_feature_shape=False)
context.run(schema_gen)


ExecutionResult(
    component_id: SchemaGen
    execution_id: 10
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [38]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'city',STRING,required,single,'city'
'delivery_perc',FLOAT,required,single,-
'net_manager',STRING,required,single,'net_manager'
'num_connections',INT,required,single,-
'perc_of_active_connections',INT,required,single,-
'purchase_area',STRING,required,single,'purchase_area'
'street',STRING,required,single,'street'
'type_conn_perc',FLOAT,required,single,-
'type_of_connection',STRING,required,single,'type_of_connection'


,Values
Domain,
'city','WERKENDAM'
'net_manager','Enexis B.V.'
'purchase_area','ENEXIS'
'street',"'Alberssteeg', 'Heuveltje', 'Koppenhof', 'Krommesteeg', 'Oudsas', 'Plein', 'Sasdijk', 'Sluisstraat', 'Van Brakelsteeg', 'Zagerij', 'Zevenhuizen'"
'type_of_connection',"'1x25', '3x25'"
'zipcode_from',"'4251AA', '4251AB', '4251AC', '4251AD', '4251AE', '4251AH', '4251AJ', '4251AK', '4251AM', '4251AP', '4251AT', '4251AV', '4251AW', '4251AX'"
'zipcode_to',"'4251AA', '4251AB', '4251AC', '4251AD', '4251AG', '4251AH', '4251AJ', '4251AL', '4251AN', '4251AR', '4251AT', '4251AV', '4251AW', '4251BB'"
